## Deep Learning based Spatio-Temporal Anomaly Detection in Videos
### Team Members : Haritha Selvakumaran, Ravi Shankar Sankara Narayanan, Vishaq Jayakumar

Date : 04/25/2024<br>
Purpose : This notebook contains the 3D CNN code which is used as a comparative study for Anomaly Detection

### Import Libraries

In [ ]:
!pip install opencv-contrib-python

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install --upgrade tensorflow keras

Defaulting to user installation because normal site-packages is not writeable
  Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)


In [ ]:
import numpy
print(numpy.__version__)

1.25.2


In [ ]:
!pip install --upgrade numpy h5py

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 5.3 MB 2.5 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.1 requires numpy<=1.24.3,>=1.22, but you have numpy 1.24.4 which is incompatible.


In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d mateohervas/dcsass-dataset

100% 1.35G/1.35G [00:18<00:00, 77.7MB/s]
100% 1.35G/1.35G [00:18<00:00, 77.1MB/s]


In [ ]:
!unzip dcsass-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: dcsass dataset/DCSASS Dataset/Robbery/Robbery142_x264.mp4/Robbery142_x264_30.mp4  
  inflating: dcsass dataset/DCSASS Dataset/Robbery/Robbery142_x264.mp4/Robbery142_x264_31.mp4  
  inflating: dcsass dataset/DCSASS Dataset/Robbery/Robbery142_x264.mp4/Robbery142_x264_4.mp4  
  inflating: dcsass dataset/DCSASS Dataset/Robbery/Robbery142_x264.mp4/Robbery142_x264_5.mp4  
  inflating: dcsass dataset/DCSASS Dataset/Robbery/Robbery142_x264.mp4/Robbery142_x264_6.mp4  
  inflating: dcsass dataset/DCSASS Dataset/Robbery/Robbery142_x264.mp4/Robbery142_x264_7.mp4  
  inflating: dcsass dataset/DCSASS Dataset/Robbery/Robbery142_x264.mp4/Robbery142_x264_8.mp4  
  inflating: dcsass dataset/DCSASS Dataset/Robbery/Robbery142_x264.mp4/Robbery142_x264_9.mp4  
  inflating: dcsass dataset/DCSASS Dataset/Robbery/Robbery145_x264.mp4/Robbery145_x264_0.mp4  
  inflating: dcsass dataset/DCSASS Dataset/Robbery/Robbery145_x264.mp4/Robbery145_x264_1.mp4

In [ ]:
import cv2
import pandas as pd
import os
import numpy as np
import random
import h5py
import csv
import tensorflow as tf
#from tensorflow import keras
from keras.utils import Sequence
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"harithaselvakumaran","key":"ff8a8b350d1dfbbba2806c10bbaa86cc"}'}

### Creation of CSV file with frame location and Labels

In [ ]:
# Define the base directory and labels directory
base_dir = 'DCSASS Dataset/'  # Replace with your base directory
labels_dir = 'DCSASS Dataset/Labels/'  # Replace with your labels directory

# Get the categories
categories = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d)) and d != 'Labels']

# Define the path to the output CSV file
csv_output_path = 'video_paths_and_labels.csv'

# Open the output CSV file
with open(csv_output_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # Write the header row
    writer.writerow(['video_path', 'label'])

    # Iterate over the categories
    for category in categories:
        category_path = os.path.join(base_dir, category)
        csv_file_path = os.path.join(labels_dir, f'{category}.csv')

        labels_df = pd.read_csv(csv_file_path)

        # Iterate over the video folders
        for video_folder in os.listdir(category_path):
            video_folder_path = os.path.join(category_path, video_folder)

            if not os.path.isdir(video_folder_path):
                continue

            # Iterate over the video files
            for video_file in os.listdir(video_folder_path):
                video_path = os.path.join(video_folder_path, video_file)

                video_base_name = os.path.splitext(video_file)[0]

                label_row = labels_df[labels_df.iloc[:, 0].str.match(video_base_name)]

                if not label_row.empty:
                    label = category if label_row.iloc[0,2] == 1 else 'Normal'

                    video_path = video_path.replace('\\', '/')
                    # Write the video path and label to the CSV file
                    writer.writerow([video_path, label])

In [ ]:
df = pd.read_csv('video_paths_and_labels.csv')

In [ ]:
df

,video_path,label
0,DCSASS Dataset/Assault/Assault049_x264.mp4/Ass...,Assault
1,DCSASS Dataset/Assault/Assault049_x264.mp4/Ass...,Normal
2,DCSASS Dataset/Assault/Assault049_x264.mp4/Ass...,Normal
3,DCSASS Dataset/Assault/Assault049_x264.mp4/Ass...,Normal
4,DCSASS Dataset/Assault/Assault049_x264.mp4/Ass...,Assault
...,...,...
16575,DCSASS Dataset/Burglary/Burglary035_x264.mp4/B...,Normal
16576,DCSASS Dataset/Burglary/Burglary035_x264.mp4/B...,Normal
16577,DCSASS Dataset/Burglary/Burglary035_x264.mp4/B...,Burglary
16578,DCSASS Dataset/Burglary/Burglary035_x264.mp4/B...,Normal


In [ ]:
df['label'].value_counts()

label
Normal           9490
Robbery          1891
Burglary         1009
Stealing          965
Abuse             568
Arrest            497
RoadAccidents     461
Vandalism         404
Arson             314
Shooting          304
Assault           278
Explosion         158
Shoplifting       155
Fighting           86
Name: count, dtype: int64

In [ ]:
df_train, df_val = train_test_split(df, test_size = 0.2, random_state = 42)

### Modify classweights to address imbalance towards the dataset

In [ ]:
le = LabelEncoder()
le.fit(df_train['label'])
df_train['label'] = le.fit_transform(df_train['label'])

In [ ]:
df_train['label'].value_counts()

label
7     7577
9     1520
4      812
12     767
0      448
1      404
8      360
13     334
2      262
10     251
3      213
11     127
5      124
6       65
Name: count, dtype: int64

In [ ]:
le = LabelEncoder()
le.fit(df_val['label'])
df_val['label'] = le.transform(df_val['label'])

In [ ]:
df_val['label'].value_counts()

label
7     1913
9      371
12     198
4      197
0      120
8      101
1       93
13      70
3       65
10      53
2       52
5       34
11      28
6       21
Name: count, dtype: int64

In [ ]:
df_train

,video_path,label
14014,DCSASS Dataset/Robbery/Robbery071_x264.mp4/Rob...,9
14459,DCSASS Dataset/Robbery/Robbery077_x264.mp4/Rob...,9
13707,DCSASS Dataset/Robbery/Robbery087_x264.mp4/Rob...,9
14100,DCSASS Dataset/Robbery/Robbery050_x264.mp4/Rob...,9
13976,DCSASS Dataset/Robbery/Robbery106_x264.mp4/Rob...,7
...,...,...
11284,DCSASS Dataset/Shooting/Shooting034_x264.mp4/S...,10
11964,DCSASS Dataset/Robbery/Robbery139_x264.mp4/Rob...,9
5390,DCSASS Dataset/Shoplifting/Shoplifting054_x264...,7
860,DCSASS Dataset/Arson/Arson014_x264.mp4/Arson01...,2


In [ ]:
df_val

,video_path,label
15188,DCSASS Dataset/Burglary/Burglary013_x264.mp4/B...,4
15735,DCSASS Dataset/Burglary/Burglary069_x264.mp4/B...,4
13771,DCSASS Dataset/Robbery/Robbery041_x264.mp4/Rob...,9
3165,DCSASS Dataset/Arrest/Arrest009_x264.mp4/Arres...,7
15688,DCSASS Dataset/Burglary/Burglary027_x264.mp4/B...,4
...,...,...
13090,DCSASS Dataset/Robbery/Robbery118_x264.mp4/Rob...,7
14067,DCSASS Dataset/Robbery/Robbery009_x264.mp4/Rob...,9
5479,DCSASS Dataset/Shoplifting/Shoplifting022_x264...,7
3703,DCSASS Dataset/Abuse/Abuse007_x264.mp4/Abuse00...,7


In [ ]:
#df_train['label'] = df_train['label'].apply(lambda x: to_categorical(x, num_classes=14))
#df_val['label'] = df_val['label'].apply(lambda x: to_categorical(x, num_classes=14))

<ipython-input-41-de108fbeb54d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['label'] = df_train['label'].apply(lambda x: to_categorical(x, num_classes=14))
<ipython-input-41-de108fbeb54d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['label'] = df_val['label'].apply(lambda x: to_categorical(x, num_classes=14))


In [ ]:
labels_one_hot = to_categorical(df_train['label'], num_classes=14)
labels_one_hot_val = to_categorical(df_val['label'], num_classes=14)

In [ ]:
len(df_train), len(df_val)

(13264, 3316)

### Process Video

In [ ]:
def process_video(video_path, clip_length=60):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (64, 64))
        frame = frame / 255
        frames.append(frame)
    frames = np.array(frames)
    if len(frames) < clip_length:
        padding = np.zeros((clip_length - len(frames), 64, 64, 3))
        frames = np.concatenate((frames, padding))
    clips = []
    for i in range(0, len(frames), clip_length):
        clip = frames[i:i+clip_length]
        if len(clip) < clip_length:
            padding = np.zeros((clip_length - len(clip), 64, 64, 3))
            clip = np.concatenate((clip, padding))
        clips.append(clip)
    return np.array(clips)

### Generators to get the frames and pass it to the model in batches

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, df, batch_size=32, dim=(64, 64), n_channels=3, n_frames=60, shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.n_frames = n_frames
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.df['video_path'].values[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, self.n_frames, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, 14), dtype=int)
        for i, ID in enumerate(list_IDs_temp):
            video_clips = process_video(ID)
            for clip in video_clips:
                X[i,] = clip
                label = self.df[self.df['video_path'] == ID]['label'].values[0]
                y[i] = to_categorical(label, num_classes=14)
        return X, y

In [ ]:
params = {'dim': (64, 64),
          'batch_size': 64,
          'n_channels': 3,
          'n_frames': 60,
          'shuffle': True}

In [ ]:
training_generator = DataGenerator(df_train, **params)
validation_generator = DataGenerator(df_val, **params)

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


### Model Building

In [ ]:
n_classes = 14
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = Sequential()
    model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(params['n_frames'], *params['dim'], params['n_channels'])))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Training the model

In [ ]:
history = model.fit(training_generator, validation_data=validation_generator, epochs=10, verbose = 1)

Epoch 1/10
207/207 [==============================] - 647s 3s/step - loss: 1.4739 - accuracy: 0.6200 - val_loss: 0.9260 - val_accuracy: 0.7215
Epoch 2/10
207/207 [==============================] - 643s 3s/step - loss: 0.6741 - accuracy: 0.7634 - val_loss: 0.6685 - val_accuracy: 0.7616
Epoch 3/10
207/207 [==============================] - 644s 3s/step - loss: 0.4871 - accuracy: 0.8113 - val_loss: 0.5178 - val_accuracy: 0.8015
Epoch 4/10
207/207 [==============================] - 644s 3s/step - loss: 0.3928 - accuracy: 0.8443 - val_loss: 0.5101 - val_accuracy: 0.8162
Epoch 5/10
207/207 [==============================] - 640s 3s/step - loss: 0.3314 - accuracy: 0.8721 - val_loss: 0.4779 - val_accuracy: 0.8211
Epoch 6/10
207/207 [==============================] - 640s 3s/step - loss: 0.2848 - accuracy: 0.8885 - val_loss: 0.5161 - val_accuracy: 0.8211
Epoch 7/10
207/207 [==============================] - 645s 3s/step - loss: 0.2504 - accuracy: 0.9017 - val_loss: 0.4980 - val_accuracy: 0.8315